# Notebook for the preprocessing and EDA steps
Here we progress with the feature engineering and baseline model steps.

In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from collections import Counter
pd.set_option('display.max_rows', 30)
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score

## Upload the dataset

In [2]:
# Read pickle file
wok_dir = Path.cwd() / 'datasets/'
filename = 'facies_dataset_preprocess.parquet'
path_file = wok_dir / filename
data = pd.read_parquet(path_file) # Read file

data.head()

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,FaciesLabels
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000,FSiS
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979,FSiS
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957,FSiS
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936,FSiS
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915,FSiS


In [3]:
# Check dtype - it matches with preprocessing step
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4149 entries, 0 to 4148
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Facies        4149 non-null   int64   
 1   Formation     4149 non-null   object  
 2   Well Name     4149 non-null   category
 3   Depth         4149 non-null   float64 
 4   GR            4149 non-null   float64 
 5   ILD_log10     4149 non-null   float64 
 6   DeltaPHI      4149 non-null   float64 
 7   PHIND         4149 non-null   float64 
 8   PE            4149 non-null   float64 
 9   NM_M          4149 non-null   int64   
 10  RELPOS        4149 non-null   float64 
 11  FaciesLabels  4149 non-null   object  
dtypes: category(1), float64(7), int64(2), object(2)
memory usage: 361.1+ KB


## Feature engineering
First step is always feature extraction from categorical features such as `Formation` that we need to convert to `numerical` feature

In [4]:
data['Formation_num'] = LabelEncoder().fit_transform(data['Formation'].astype(str)) + 1

data.head()

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,FaciesLabels,Formation_num
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000,FSiS,2
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979,FSiS,2
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957,FSiS,2
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936,FSiS,2
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915,FSiS,2


### Select a well for testing phase
Our model will not see the data from `Well Name == KIMZEY A` during the training process. We use the data from this well as test, the rest it is for tranining.

In [5]:
test_data = data[data['Well Name'] == 'KIMZEY A']
data_fe = data[data['Well Name'] != 'KIMZEY A'] # Data for feature engineering

In [6]:
data_fe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3710 entries, 0 to 4148
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Facies         3710 non-null   int64   
 1   Formation      3710 non-null   object  
 2   Well Name      3710 non-null   category
 3   Depth          3710 non-null   float64 
 4   GR             3710 non-null   float64 
 5   ILD_log10      3710 non-null   float64 
 6   DeltaPHI       3710 non-null   float64 
 7   PHIND          3710 non-null   float64 
 8   PE             3710 non-null   float64 
 9   NM_M           3710 non-null   int64   
 10  RELPOS         3710 non-null   float64 
 11  FaciesLabels   3710 non-null   object  
 12  Formation_num  3710 non-null   int64   
dtypes: category(1), float64(7), int64(3), object(2)
memory usage: 380.8+ KB


In order to see the impact of feature engineering we should build a baseline model to assist this task. Then, we can compare the baseline results with the ones from ML models with feature engineering.

# Baseline model
The baseline model should be a linear ML method (for regression is a linear regression and for classification is a logistic regression). This baseline model is built **without** feature extration or similar kind.

In [7]:
# Split the dataset into X and Y
X = data_fe[['Depth', 'GR', 'ILD_log10','DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS', 'Formation_num']]
y = data_fe['Facies']

In [8]:
# Import sklearn Log_reg
from sklearn.linear_model import LogisticRegression

LOG = LogisticRegression(solver='liblinear')

## Create the model
Obviously, we will implement the model with cross validation and examine its performance.

In [9]:
model = LOG
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f' % (scores.mean()))

Accuracy: 0.573


### Insights
We can say the following:
 - Now we can compare the feature engineering step, and explore feautre extraction for improving the model `Accuracy`.
 - Other metrics can be used instead the `Accuracy`
 - There are many approaches while we will use some transforms for chaining the distribution of the input variables such as Quantile Transformer and KBins Discretizer. Then, will remove linear dependencies between the input variables using PCA and TruncatedSVD.
 - We will use the `sklearn.Pipeline` to define a list of transforms to perform the feature extraction. This will create a dataset with lots of feature columns while we need to reduce dimensionality to faster and better performance. Finally, Recursive Feature Elimination, or RFE, technique can be used to select the most relevant features. We select 30 features.

## Build pipeline 
We can use the baseline model together with **imputers** and **transformers** for feature extraction that perform feature engineering, before moving to more advance ML methods. Here, we will build a pipeline with different transformers to see how well they perform.

In [15]:
# Import the necessary classes from sklearn
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

# Build the transformers
transforms = list()
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))
# Wrapper the transformrs
featurizer = FeatureUnion(transforms)
# Define the feature selection
rfe = RFE(estimator=LogisticRegression(solver='liblinear'), n_features_to_select=30)
# Model creation
model = LOG

# Pipeline to build the sequence (as steps)
steps = list() # Attach all steps in sequence
steps.append(('fu', featurizer))
steps.append(('rfe', rfe))
steps.append(('ml', model))
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1) # Cross validation for the training

# Training model
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# Print average score
print('Accuracy: %.3f' % (scores.mean()))

Accuracy: 0.614


### Conclusions
 - Improved `Accuracy` in 0.04 absolute value which is good for the model
 - Accuracy improvement shows that feature engineering can be useful approach when we are dealing with limited features in dataset
 - Avoiding as well the implementation of complex models with this simple engineering skill

## Performance with imbalance datasets
We are dealing with imbalance dataset where some classes have few datapoints, and this can affect the `Accuracy` of the method. In imbalanced datasets, we can use the resampling technique to add some more data points to increase members of minority groups. This can be helpful whenever minority label targets have special importance such as credit card or payment fraud detection (with typical less than 0.1 percent of fraud). For that, we can use **SMOTE** that is a very popular technique for oversampling the *minority* class (or classes).

In [14]:
# Import the SMOTE technique
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_sm1 , y_sm1 = smote.fit_resample(X,y)
X_sm , y_sm = X_sm1 , y_sm1  # keep for future plotting an comparison

print("Before SMOTE: ", Counter(y))
print("After SMOTE: ", Counter(y_sm))

Before SMOTE:  Counter({2: 855, 3: 706, 8: 596, 6: 531, 1: 259, 5: 243, 4: 228, 9: 178, 7: 114})
After SMOTE:  Counter({3: 855, 2: 855, 8: 855, 6: 855, 7: 855, 4: 855, 5: 855, 9: 855, 1: 855})


Now we have all 9 classes completely (and equally) balanced. It is worthy to investigate the results of the baseline model (without feature engineering) with this balanced dataset.

In [17]:
# Transform the data to Standard distribution (mu=0 and std=1)
scaler = StandardScaler()
X_sm = scaler.fit_transform(X_sm)

# Re-run the baseline model with the balanced dataset
scores = cross_val_score(model, X_sm, y_sm, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f' % (scores.mean()))

Accuracy: 0.613


This model reached the (almost) same `Accuracy` as the model before using the `pipeline` with preprocessors and transformers. We could even test the **SMOTE** with feature engineering.

In [18]:
# Re-run the pipeline with the feature extraction
scores = cross_val_score(pipeline, X_sm, y_sm, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f' % (scores.mean()))

Accuracy: 0.640


Again, `Accuracy` increases 0.03% but in multi-class classification, `Accuracy` is not the best evaluation metrics. We will cover others in next parts.

## Feature importance
TBD

# Save Feat eng dataset
Let us save the balanced dataset with **SMOTE** since it presents good results in terms of `Accuracy`. We will use for later implementations.

In [34]:
# Convert X_sm, y_sm to dataframes
x_sm_df = pd.DataFrame(X_sm, columns=['Depth', 'GR', 'ILD_log10','DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS', 'Formation_num'])
y_sm_df = pd.DataFrame(y_sm, columns=['Facies'])
# Merge the X, y
data_sm = pd.concat([x_sm_df, y_sm_df], axis=1)

In [35]:
# Save on parquet file (spark format)
import pyarrow as pa
import pyarrow.parquet as pq

# Convert DataFrame to Apache Arrow Table
data_pq = pa.Table.from_pandas(data_sm)
# Parquet with Brotli compression
pq.write_table(data_pq, 'facies_dataset_SMOTE.parquet')